<문장임베딩 및 유사도 분석 참조 사이트>
https://github.com/BM-K/KoSentenceBERT_SKT
https://github.com/SKTBrain/KoBERT
https://www.infoking.site/52
https://www.sbert.net/docs/pretrained_models.html

In [2]:
import numpy as np
import pandas as pd

In [19]:
df_road_seoul = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\Dataset\\202109_주소DB_전체분\\개선_도로명코드_전체분.txt', sep = '|', encoding = 'cp949', header = None)
df_road_seoul.columns = ['도로명코드', '도로명', '도로명로마자', '읍면동일련번호', '시도명', '시도명로마자', '시군구명', '시군구로마자', '읍면동명', '읍면동로마자', '읍면동구분', '읍면동코드', '사용여부', '변경사유', '변경이력정보', '고시일자', '말소일자']
df_road_seoul.drop(['도로명로마자', '읍면동일련번호','시도명로마자', '시군구로마자', '읍면동로마자', '읍면동구분', '읍면동코드', '사용여부', '변경사유', '변경이력정보', '고시일자', '말소일자'], axis = 1, inplace = True)
df_road_seoul['읍면동명'] = df_road_seoul['읍면동명'].apply(lambda x: '0' if str(x) == 'nan' else x)
print(df_road_seoul.dtypes)
df_road_seoul.head()

도로명코드     int64
도로명      object
시도명      object
시군구명     object
읍면동명     object
dtype: object


,도로명코드,도로명,시도명,시군구명,읍면동명
0,111102005001,세종대로,서울특별시,종로구,0
1,111102005001,세종대로,서울특별시,종로구,세종로
2,111102005001,세종대로,서울특별시,종로구,종로1가
3,111102100001,삼일대로,서울특별시,종로구,0
4,111102100001,삼일대로,서울특별시,종로구,운니동


In [23]:
df_juso_seoul = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\Dataset\\202109_주소DB_전체분\\주소_서울특별시.txt', sep = '|', encoding = 'cp949', header = None)
df_juso_seoul.columns = ['관리번호', '도로명코드', '읍면동일련번호', '지하여부', '건물(본번)', '건물(부번)', '기초구역번호', '변경사유코드', '고시일자', '변경전도로명주소', '상세주소부여여부']
df_juso_seoul.drop(['읍면동일련번호', '지하여부', '기초구역번호', '변경사유코드', '고시일자', '변경전도로명주소', '상세주소부여여부'], axis = 1, inplace = True)
df_juso_seoul['건물(본번)'] = df_juso_seoul['건물(본번)'].apply(lambda x: str(x))
df_juso_seoul['건물(부번)'] = df_juso_seoul['건물(부번)'].apply(lambda x: str(x))
df_juso_seoul['건물번호'] = df_juso_seoul['건물(본번)'] + '-' + df_juso_seoul['건물(부번)']
df_juso_seoul['건물번호'] = df_juso_seoul['건물번호'].apply(lambda x: str(x).replace('-0', ''))
df_juso_seoul

,관리번호,도로명코드,건물(본번),건물(부번),건물번호
0,1111011900102150000000001,111102005001,145,2,145-2
1,1111011900100010080000001,111102005001,147,2,147-2
2,1111012000101500000000347,111102005001,149,0,149
3,1111011900101390008017955,111102005001,152,0,152
4,1111011900102020001028380,111102005001,159,0,159
...,...,...,...,...,...
540290,1174011000101590000000001,117404858126,49,0,49
540291,1174011000101550008000001,117404858126,50,0,50
540292,1174011000101580003000001,117404858126,57,0,57
540293,1174011000101760002000001,117404858126,59,0,59


In [37]:
df_road_seoul['도로명코드'].value_counts()

111403101011    29
481252019003    23
271103007001    21
281103149064    21
111103100010    20
                ..
411154328101     1
411154328086     1
415004855515     1
415004855516     1
501304860169     1
Name: 도로명코드, Length: 170109, dtype: int64

In [39]:
df_road_seoul = df_road_seoul.drop_duplicates(['도로명코드'], keep = 'first')

In [62]:
df_seoul = pd.merge(df_juso_seoul, df_road_seoul, left_on='도로명코드', right_on='도로명코드', how='outer')
df_seoul['도로명주소'] = df_seoul['시도명'] + ' ' + df_seoul['시군구명'] + ' ' + df_seoul['도로명'] + ' ' + df_seoul['건물번호']
df_seoul.dropna(subset = ['관리번호'], inplace = True)
df_seoul.drop(['건물(본번)', '건물(부번)', '시도명', '시군구명', '읍면동명', '도로명코드'], axis = 1, inplace = True)
df_seoul.head()

,관리번호,건물번호,도로명,도로명주소
0,1111011900102150000000001,145-2,세종대로,서울특별시 종로구 세종대로 145-2
1,1111011900100010080000001,147-2,세종대로,서울특별시 종로구 세종대로 147-2
2,1111012000101500000000347,149,세종대로,서울특별시 종로구 세종대로 149
3,1111011900101390008017955,152,세종대로,서울특별시 종로구 세종대로 152
4,1111011900102020001028380,159,세종대로,서울특별시 종로구 세종대로 159


In [51]:
df_jibun_seoul = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\Dataset\\202109_주소DB_전체분\\지번\\지번_서울특별시.txt', sep = '|', encoding = 'cp949', header = None)
df_jibun_seoul.columns = ['관리번호', '일련번호', '법정동코드', '시도명', '시군구명', '법정읍면동명', '법정리명', '산여부', '지번(본번)', '지번(부번)', '대표여부']
df_jibun_seoul

,관리번호,일련번호,법정동코드,시도명,시군구명,법정읍면동명,법정리명,산여부,지번(본번),지번(부번),대표여부
0,1111010100100010000030843,1,1111010100,서울특별시,종로구,청운동,NaN,0,1,0,1
1,1111010100100030000030936,1,1111010100,서울특별시,종로구,청운동,NaN,0,3,0,1
2,1111010100100030065030970,1,1111010100,서울특별시,종로구,청운동,NaN,0,3,100,1
3,1111010100100030065030970,2,1111010100,서울특별시,종로구,청운동,NaN,0,3,65,0
4,1111010100100030150030938,1,1111010100,서울특별시,종로구,청운동,NaN,0,3,150,1
...,...,...,...,...,...,...,...,...,...,...,...
628124,1174011000200220014000001,1,1174010300,서울특별시,강동구,상일동,NaN,1,73,0,1
628125,1174011000200220036000002,1,1174010300,서울특별시,강동구,상일동,NaN,1,77,23,1
628126,1174011000200220036000002,2,1174010300,서울특별시,강동구,상일동,NaN,0,554,15,0
628127,1174011000200220036000002,3,1174010300,서울특별시,강동구,상일동,NaN,0,554,16,0


In [59]:
df_train = df_jibun_seoul.drop(['법정동코드', '일련번호', '산여부', '대표여부'], axis = 1).copy()
df_train['법정리명'] = df_train['법정리명'].apply(lambda x: '0' if str(x) == 'nan' else x)
df_train['지번(본번)'] = df_train['지번(본번)'].apply(lambda x: str(x))
df_train['지번(부번)'] = df_train['지번(부번)'].apply(lambda x: str(x))
df_train['지번주소'] = df_train['시도명'] + ' ' + df_train['시군구명'] + ' ' + df_train['법정읍면동명'] + ' ' + df_train['법정리명'] + ' ' + df_train['지번(본번)'] + '-' + df_train['지번(부번)']
df_train['지번주소'] = df_train['지번주소'].apply(lambda x: str(x).replace('0 ', ''))
df_train['지번주소'] = df_train['지번주소'].apply(lambda x: str(x).replace('-0', ''))
df_train

,관리번호,시도명,시군구명,법정읍면동명,법정리명,지번(본번),지번(부번),지번주소
0,1111010100100010000030843,서울특별시,종로구,청운동,0,1,0,서울특별시 종로구 청운동 1
1,1111010100100030000030936,서울특별시,종로구,청운동,0,3,0,서울특별시 종로구 청운동 3
2,1111010100100030065030970,서울특별시,종로구,청운동,0,3,100,서울특별시 종로구 청운동 3-100
3,1111010100100030065030970,서울특별시,종로구,청운동,0,3,65,서울특별시 종로구 청운동 3-65
4,1111010100100030150030938,서울특별시,종로구,청운동,0,3,150,서울특별시 종로구 청운동 3-150
...,...,...,...,...,...,...,...,...
628124,1174011000200220014000001,서울특별시,강동구,상일동,0,73,0,서울특별시 강동구 상일동 73
628125,1174011000200220036000002,서울특별시,강동구,상일동,0,77,23,서울특별시 강동구 상일동 77-23
628126,1174011000200220036000002,서울특별시,강동구,상일동,0,554,15,서울특별시 강동구 상일동 554-15
628127,1174011000200220036000002,서울특별시,강동구,상일동,0,554,16,서울특별시 강동구 상일동 554-16


In [71]:
df_total = pd.merge(df_seoul, df_train, left_on='관리번호', right_on='관리번호', how='outer')
df_total.drop(['건물번호', '시도명', '시군구명', '법정읍면동명', '지번(본번)', '지번(부번)', '법정리명', '도로명'], axis = 1, inplace = True)
df_total

,관리번호,도로명주소,지번주소
0,1111011900102150000000001,서울특별시 종로구 세종대로 145-2,서울특별시 종로구 세종로 215
1,1111011900100010080000001,서울특별시 종로구 세종대로 147-2,서울특별시 종로구 세종로 1-80
2,1111012000101500000000347,서울특별시 종로구 세종대로 149,서울특별시 종로구 세종로 211
3,1111012000101500000000347,서울특별시 종로구 세종대로 149,서울특별시 종로구 신문로1가 150
4,1111012000101500000000347,서울특별시 종로구 세종대로 149,서울특별시 중구 태평로1가 68
...,...,...,...
628124,1174011000101550008000001,서울특별시 강동구 가래여울길 50,서울특별시 강동구 강일동 154
628125,1174011000101580003000001,서울특별시 강동구 가래여울길 57,서울특별시 강동구 강일동 158-3
628126,1174011000101580003000001,서울특별시 강동구 가래여울길 57,서울특별시 강동구 강일동 158-5
628127,1174011000101760002000001,서울특별시 강동구 가래여울길 59,서울특별시 강동구 강일동 176-2


In [68]:
df_total['지번주소'].isnull().sum(), df_total['도로명주소'].isnull().sum() 

(0, 0)

In [69]:
df_total['지번주소'].value_counts()

서울특별시 노원구 상계동 161-13     257
서울특별시 노원구 상계동 65-36      127
서울특별시 노원구 상계동 5-15       100
서울특별시 노원구 상계동 5-16        97
서울특별시 관악구 신림동 808-498     95
                        ... 
서울특별시 성북구 석관동 64-24        1
서울특별시 성북구 석관동 64-4         1
서울특별시 성북구 석관동 64-8         1
서울특별시 성북구 석관동 64-15        1
서울특별시 강동구 강일동 177          1
Name: 지번주소, Length: 600470, dtype: int64

In [70]:
df_total['도로명주소'].value_counts()

서울특별시 마포구 대흥로24길 24    513
서울특별시 중구 청계천로 246      449
서울특별시 동작구 사당로 90       302
서울특별시 강남구 양재대로 478     210
서울특별시 강서구 개화동로8길 38    202
                      ... 
서울특별시 강북구 노해로7길 12       1
서울특별시 강북구 노해로7길 10       1
서울특별시 강북구 노해로7길 7-6      1
서울특별시 강북구 노해로7길 7-5      1
서울특별시 강동구 가래여울길 61       1
Name: 도로명주소, Length: 540250, dtype: int64

In [73]:
df_total_train = df_total.drop_duplicates(['지번주소'], keep = 'first').copy()
df_total_train = df_total_train.drop_duplicates(['도로명주소'], keep = 'first')

In [77]:
df_total_train['지번주소'].value_counts()

서울특별시 종로구 세종로 215       1
서울특별시 구로구 구로동 709-3     1
서울특별시 구로구 구로동 710-16    1
서울특별시 구로구 구로동 718-39    1
서울특별시 구로구 구로동 710-4     1
                       ..
서울특별시 성북구 종암동 54-368    1
서울특별시 성북구 종암동 54-374    1
서울특별시 성북구 종암동 54-438    1
서울특별시 성북구 종암동 54-388    1
서울특별시 강동구 강일동 177       1
Name: 지번주소, Length: 521434, dtype: int64

In [76]:
df_total_train['도로명주소'].value_counts()

서울특별시 종로구 세종대로 145-2       1
서울특별시 구로구 구로동로17길 31-4     1
서울특별시 구로구 구로동로17길 31-18    1
서울특별시 구로구 구로동로17길 31-17    1
서울특별시 구로구 구로동로17길 31-16    1
                          ..
서울특별시 성북구 종암로9다길 31        1
서울특별시 성북구 종암로9다길 30        1
서울특별시 성북구 종암로9다길 29-1      1
서울특별시 성북구 종암로9다길 29        1
서울특별시 강동구 가래여울길 61         1
Name: 도로명주소, Length: 521434, dtype: int64

In [82]:
df_total_train.reset_index(drop = True, inplace = True)
df_total_train.to_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\Dataset\\주소변환완료.csv', index = False)
#  ------------------------------- 데이터 전처리 완료 ---------------------------------

In [83]:
df = pd.read_csv('C:\\Workspace\\python\\빅데이터 지능형서비스 개발 팀프로젝트\\PJTJ4U\\조정범\\Data\\Dataset\\주소변환완료.csv')
df

,관리번호,도로명주소,지번주소
0,1111011900102150000000001,서울특별시 종로구 세종대로 145-2,서울특별시 종로구 세종로 215
1,1111011900100010080000001,서울특별시 종로구 세종대로 147-2,서울특별시 종로구 세종로 1-80
2,1111012000101500000000347,서울특별시 종로구 세종대로 149,서울특별시 종로구 세종로 211
3,1111011900101390008017955,서울특별시 종로구 세종대로 152,서울특별시 종로구 세종로 139-8
4,1111011900102020001028380,서울특별시 종로구 세종대로 159,서울특별시 종로구 세종로 202-1
...,...,...,...
521429,1174011000101590000000001,서울특별시 강동구 가래여울길 49,서울특별시 강동구 강일동 159
521430,1174011000101550008000001,서울특별시 강동구 가래여울길 50,서울특별시 강동구 강일동 155-8
521431,1174011000101580003000001,서울특별시 강동구 가래여울길 57,서울특별시 강동구 강일동 158-3
521432,1174011000101760002000001,서울특별시 강동구 가래여울길 59,서울특별시 강동구 강일동 176-2


In [92]:
# Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks
# 출처 : https://github.com/UKPLab/sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

In [93]:
sentences = [juso for juso in df['지번주소']] + [juso for juso in df['도로명주소']]
len(sentences), len(set(sentences)) # 중복값 확인

(1042868, 1042868)

In [94]:
# 지번주소 임베딩화
sentence_embeddings = model.encode(sentences[:100], convert_to_tensor = True)
sentence_embeddings.shape

torch.Size([100, 768])

In [98]:
input = '서울시 좀로구 경임동 20-1'
input_embeddings = model1.encode(input, convert_to_tensor = True)

In [99]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_scores = util.pytorch_cos_sim(sentence_embeddings, input_embeddings)
cosine_scores_list = cosine_scores.tolist()
cosine_scores_list.index(max(cosine_scores_list))
print(f"'{input}'과 가장 유사한 주소(유사도 : {max(cosine_scores_list)} : '{sentences[cosine_scores_list.index(max(cosine_scores_list))]}'")

'서울시 좀로구 경임동 20-1'과 가장 유사한 주소(유사도 : [0.9213595986366272] : '서울특별시 종로구 경운동 21-1'
